# Análisis de los datos provenientes del PDM

Este `jupyter` y los relacionados contiene el análisis y el proceso que permite la homologación/conversión entre los datos en formato `.csv` provenientes de la base de datos de los Planes de Desarrollo Municipal (PDM) de Honduras y el modelo de datos de Fonsagua.

En concreto en este `jupyter` se analizan los códigos de comunidad proporcionados en los PDM y se realizan varios cruces con los datos del INE y de Comunidades existentes en la base de datos de Fonsagua, para obtener una metodología de como emparejarlos.

Partimos del fichero `pdm.xlsx` generado en `nasmar_comprobaciones_csv_pdm.ipynb`

In [1]:
# Configuración básica de jupyter, pandas, ...
# e importación de librerías que necesitaremos durante el análisis

# https://medium.com/@1522933668924/using-matplotlib-in-jupyter-notebooks-comparing-methods-and-some-tips-python-c38e85b40ba1
# Para hacer los mapas interactivos

# %matplotlib widget 
%matplotlib inline 
import numpy as np
from IPython.display import display, HTML, Markdown
# Ejemplos de uso
# display(HTML('<h2 style="color:red;">Subtitulo "YYY - NOMBRE"</h2>'))
# display(Markdown("# Título principal"))
# display(HTML(df.describe().to_html()))  # muestra todos los datos
import pandas
from ietl import pandas_utils

from ietl.geopandas_utils import (
    get_dataframe_from_spatialite_table,
    plot_polygon_labels,
)
import geopandas as gpd

pandas.options.mode.use_inf_as_na = True
# http://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
# pandas.set_option('display.max_rows', 500)
# pandas.set_option('display.max_columns', 500)


# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
# Si se está trabajando con módulos externos usar %autoreload 2 evita tener que
# recargar a mano. %autoreload 0 lo desactiva
%load_ext autoreload
%autoreload 2



# Ejecuta el .ipyndb referenciando como si fuera en este propio Lab
# Todas las variables del otro pasan a estar disponible en este
# Cualquier salida por pantalla del otro se muestra en los resultados 
# de la celda donde se ejecuta el %run
# if __name__ == '__main__' and '__file__' not in globals():
#     print("Este código sólo se ejecuta cuando lo llamo directamente y no a través de un `%run`")
# %run nasmar_cartografia.ipynb

In [ ]:
from nasmar import common_left_field, common_right_field
import nasmar

sqlite_path = "191208_fonsagua_todos_municipios.sqlite"
excel_datos_pdm_path = "pdm.xlsx"

In [2]:
pdm = pandas.read_excel(excel_datos_pdm_path, sheet_name="Datos_PDM", dtype = {"codigo_comunidad": object, "sort_code": object})
pdm.set_index("codigo_comunidad")  # En dos pasos para poder setearlo como object

,nombre_comunidad,nombre_municipio,total_viviendas,total_familias,total_personasv,total_sectorpri,total_sectorsec,total_sectorter,total_sectorcom,total_sectorind,...,total_hombres5a18,total_mujeres18a60,total_hombres18a60,total_adultos,total_ancianosm60,total_ancianasm60,total_ancianos,meta_nombre_comunidad_mal,nombre_comunidad_sin_codigo,sort_code
codigo_comunidad,,,,,,,,,,,,,,,,,,,,,
060601001001,001 - BO. EL CENTRO,EL TRIUNFO,17,22,69,0,0,0,0,0,...,13,16,21,39,2,1,3,False,BO. EL CENTRO,060601001
060601001002,002 - COL. BELLA HORIZONTE,EL TRIUNFO,71,77,301,4,0,0,2,0,...,43,67,88,165,10,8,18,False,COL. BELLA HORIZONTE,060601001
060601001003,003 - BO. EL TOLOLO,EL TRIUNFO,149,156,605,9,32,26,19,3,...,76,135,172,325,28,34,62,False,BO. EL TOLOLO,060601001
060601001004,004 - BO. BRISAS DEL RIO GUALE,EL TRIUNFO,71,88,289,17,2,1,4,1,...,45,57,87,154,11,19,30,False,BO. BRISAS DEL RIO GUALE,060601001
060601001005,005 - BO. EL CALVARIO,EL TRIUNFO,44,51,209,3,3,1,20,0,...,39,43,55,108,7,6,13,False,BO. EL CALVARIO,060601001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170910005000,005 - LAS LOMAS,SAN LORENZO,3,3,9,0,0,0,0,0,...,0,3,4,7,0,0,0,False,LAS LOMAS,170910005
170910006000,006 - LAS PILAS,SAN LORENZO,28,29,100,2,0,0,0,0,...,7,26,30,57,3,2,5,False,LAS PILAS,170910006
170910007000,007 - VOLCANCITO,SAN LORENZO,26,30,94,0,0,0,0,0,...,7,26,26,54,5,3,8,False,VOLCANCITO,170910007


In [3]:
comunidades = nasmar.get_comunidades_min()
# comunidades.to_excel("comunidades_fonsagua.xlsx")
# comunidades.to_file("comunidades.shp", encoding = 'utf-8')
# display(comunidades)

caserios = nasmar.get_caserios_nasmar()
# caserios.to_excel("caserios_ine.xlsx")
# caserios_min = caserios.drop(columns="geom")


comunidades_caserios_merge_options = {
    "left_on": "cod_comunidad",
    "right_on": "codigo",
    "how": "left",
    "indicator": False,
    "validate": "one_to_one",
    "suffixes": ("_comunidades", "_caserios"),
}

enlace_comunidades_caserios = pandas.merge(comunidades, caserios, **comunidades_caserios_merge_options)
enlace_comunidades_caserios.set_geometry("geom_comunidades")
enlace_comunidades_caserios.to_excel("enlace_comunidades_fonsagua_caserios_ine.xlsx")


cond = enlace_comunidades_caserios["codigo"].isna()
no_enlazan_comunidades_con_caserios = enlace_comunidades_caserios.loc[cond, ["cod_comunidad", "comunidad", "municipio", "n_habitantes", "tot_ancianos_comunidades", "codigo"]]
no_enlazan_comunidades_con_caserios.to_excel("no_enlazan_comunidades_fonsagua_caserios_ine.xlsx")

In [4]:
pdm_with_cod_comunidad_as_column = pdm.reset_index()
# display(pdm_with_cod_comunidad_as_column)
foo_pdm_options = {
    "left_on": "codigo",
    "right_on": "sort_code",
    "how": "outer",
    "indicator": "my_indicator",
    "validate": "one_to_many",
    "suffixes": ("_caserios", "_pdm"),
}
enlace_caserios_pdm = pandas.merge(caserios, pdm_with_cod_comunidad_as_column, **foo_pdm_options)

enlace_caserios_pdm = enlace_caserios_pdm[["codigo", "nombre", "pob_total", "categoria", "tot_ancianos", "codigo_comunidad", "nombre_comunidad", "nombre_municipio", 'total_personasv', "total_ancianos", "sort_code"]]

# display(enlace_caserios_pdm)
# enlace_caserios_pdm.to_excel("enlace_caserios_pdm.xlsx")

enlace_caserios_pdm["new_merge_code"] = enlace_caserios_pdm["codigo"]
enlace_caserios_pdm.loc[enlace_caserios_pdm["codigo"].isna(), "new_merge_code"] = enlace_caserios_pdm.loc[enlace_caserios_pdm["codigo"].isna(), "sort_code"]
# display(enlace_caserios_pdm)

total_merge_options = {
    "left_on": "new_merge_code",
    "right_on": "cod_comunidad",
    "how": "outer",
    "indicator": True,
    "validate": "many_to_one",
    "suffixes": ("_caserios_and_pdm", "_fonsagua"),
}
enlace_total = pandas.merge(enlace_caserios_pdm, comunidades, **total_merge_options)
enlace_total.drop(columns="geom", inplace=True)

# Buscamos que códigos tenemos en Fonsagua que no enlazan con nada. Y a mano buscando por nombre
# tratamos de situar con que códigos de los PDM o caserios enlazan
fonsagua_no_enlaza = enlace_total.loc[enlace_total["new_merge_code"].isna(), ["new_merge_code", "cod_comunidad", "comunidad", "municipio", "n_habitantes", "tot_ancianos_fonsagua", "_merge"]]

fonsagua_no_enlaza.loc[:, "codigo_pdm_a_mano"] = np.nan
fonsagua_no_enlaza.loc[:, "comentario"] = np.nan


# TODO. Automatizar. Búsqueda de text similarity, distancias geográficas, ...
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060601015", ["codigo_pdm_a_mano", "comentario"]] = ["060601001012", "Enlaza por nombre de la comunidad con PDM pero con uno de los códigos de barrio"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060603001", ["codigo_pdm_a_mano", "comentario"]] = ["060601001016", 'Enlaza por nombre de la comunidad con PDM pero con uno de los códigos de "barrio"']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060606029", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, 'hay una coincidencia de nombre pero para Marcovia']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060609054", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, 'ninguna coincidencia']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060702005", ["codigo_pdm_a_mano", "comentario"]] = ["060702016000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060702007", ["codigo_pdm_a_mano", "comentario"]] = ["060702022000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060716005", ["codigo_pdm_a_mano", "comentario"]] = ["060716004000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060723001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. caserios '060718006. PDM '060709027000"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060724001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Enlaza con caserios '060712002"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060725001", ["codigo_pdm_a_mano", "comentario"]] = ["060718015000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060726001", ["codigo_pdm_a_mano", "comentario"]] = ["060709026000", 'No hay caserio']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060728001", ["codigo_pdm_a_mano", "comentario"]] = ["060713003000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060729001", ["codigo_pdm_a_mano", "comentario"]] = ["'060714086000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060729002", ["codigo_pdm_a_mano", "comentario"]] = ["'060714065000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060729003", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, 'El nombre "El Chapeton" no coincide con nada']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060729004", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio = '060714032 que se corresponde con el PDM de 9 '060714032. Y también coincide con el caserio '060714059'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060730001", ["codigo_pdm_a_mano", "comentario"]] = ["060714006", "'El pdm tiene código de 9, y hay otra coincidencia en los PDM con '060703004000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060730002", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles: PDM: '060703008000. Caserio:'060709011. Caserio: '060714031'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060730004", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles: Caserio: '060714016. PDM: '060703002000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060730005", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio: 060714072, PDM: '060703009000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060730006", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio. '060714063. PDM: '060704003000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060731001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio: '060714021. PDM: '060703005000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060732001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio: '060714039, '060714023'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060733001", ["codigo_pdm_a_mano", "comentario"]] = ["060717009000", 'No enlaza con caserios']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060734001", ["codigo_pdm_a_mano", "comentario"]] = ["060714025000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060735001", ["codigo_pdm_a_mano", "comentario"]] = ["060714047000", 'Buena pinta']
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060736001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio: '060702003. PDM: '060707002000'"]
fonsagua_no_enlaza.loc[fonsagua_no_enlaza["cod_comunidad"] == "060739001", ["codigo_pdm_a_mano", "comentario"]] = [np.nan, "Posibles. Caserio: '060714076. PDM: '060704002000'"]

fonsagua_no_enlaza.to_excel("fonsagua_no_enlaza.xlsx")


# display(enlace_total)
# enlace_total.to_excel("enlace_total.xlsx")



